In [1]:
import pandas as pd
import glob, os, json
import numpy as np
from get_name_and_tag_by_ppuid import get_name_and_tag_by_ppuid
from api_token import api_token_var_list

In [119]:
json_match_data_dir = 'data/lor-match-data'
json_pattern = os.path.join(json_match_data_dir, '*.json')
file_list = glob.glob(json_pattern)

In [79]:
dfs = []
for filename in file_list:
    data = pd.read_json(filename)
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)
df_extendida = pd.concat([pd.json_normalize(df.iloc[:,0]),pd.json_normalize(df.iloc[:,1])], axis=1, join="inner")

In [7]:
df = pd.read_json('data/lor-match-data/match_data-20220212-040548.json')
# df_extendida = pd.concat([pd.json_normalize(df.iloc[:,0]),pd.json_normalize(df.iloc[:,1])], axis=1, join="inner")

In [8]:
df_list = []
for index in range(len(df.columns)):
    df_list.append(pd.DataFrame(df.iloc[:,index][index]))
    df_list[index]['API Key'] = df.columns[index]
    

In [10]:
for index in range(len(df_list)):
    df_list[index] = pd.concat([pd.json_normalize(df_list[index].iloc[:,0]),pd.json_normalize(df_list[index].iloc[:,1]), df_list[index]['API Key']], axis=1, join="inner")

In [6]:
dict_apikey_ppuids = {'{}'.format(df_list[0].iloc[0:,-1][0]) : list(np.concatenate(df_list[0].loc[(df_list[0].game_type == 'Ranked') | (df_list[0].game_mode == "SeasonalTournamentLobby")]['participants'].to_list()).flat)} 

In [12]:
dict_apikey_ppuids = {}

for index in range(len(df_list)):
    dict_apikey_ppuids['{}'.format(df_list[index].iloc[0:,-1][0])] = list(np.concatenate(df_list[index].loc[(df_list[index].game_type == 'Ranked') | (df_list[index].game_mode == "SeasonalTournamentLobby")]['participants'].to_list()).flat) 

In [14]:
def delete_duplicates_list(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

dict_apikeys_ppuids_no_duplicates = dict_apikey_ppuids.copy()
for index in range(len(df_list)):
    dict_apikeys_ppuids_no_duplicates['{}'.format(df_list[index].iloc[0:,-1][0])] = delete_duplicates_list(dict_apikeys_ppuids_no_duplicates['{}'.format(df_list[index].iloc[0:,-1][0])])

In [16]:
#check list de ppuids y separar por master y diamond+
#primero cargamos los archivos que contienen nuestra info anterior

def readData(filename):
    data = None
    with open(filename, 'r') as fp:
        data = json.load(fp)

    return data

list_master_player_ppuid = readData("data/lor-player-data/master_player_list.json")
list_highDiamond_player_ppuid = readData("data/lor-player-data/highdiamond_player_list.json")

#Cargamos base de datos del master leaderboard
df_master_leaderboard = pd.json_normalize(pd.read_json('data/master-leaderboard-data/data_americas.json').iloc[:,0])

In [ ]:
for key in dict_apikeys_ppuids_no_duplicates.keys():
    counter = 0
    print('Comenzando Loop con la API Key {}'.format(key))
    print("\n")
    for player in dict_apikeys_ppuids_no_duplicates[key]:
        counter +=1
        if player not in (list_master_player_ppuid or list_highDiamond_player_ppuid):
            try:
                ppuid_info = get_name_and_tag_by_ppuid('americas', player, key)
                if ppuid_info == 429:
                    print("\n")
                    print('Error 429 en paso {} relacionado a la API Key {}'.format(counter, key))
                    print('Terminando recolección de datos de la API Key {}'.format(key))
                    print("\n")
                elif (df_master_leaderboard.iloc[:,0] == ppuid_info['gameName']).any() == True:
                    print('Agregando {} a list_master_player_ppuid'.format(player))
                    list_master_player_ppuid.append(player)
                elif (df_master_leaderboard.iloc[:,0] == ppuid_info['gameName']).any() == False:
                    print('Agregando {} a list_highDiamond_player_ppuid'.format(player))
                    list_highDiamond_player_ppuid.append(player)
                else:
                    print(ppuid_info)
            except:
                print("\n")
                print('Hay algún otro error en el paso {} de la API Key {}....'.format(counter, key))
                print("\n")
    
    print("\n")
    print('Completado el Loop Relacionado a la API Key {}'.format(key))
    print('Reseteando el contador a cero...')
    print("\n")

In [102]:
list_ppuids_no_duplicates[40] not in (list_master_player_ppuid or list_highDiamond_player_ppuid)
ppuid_info = get_name_and_tag_by_ppuid('americas', list_ppuids_no_duplicates[40])

In [ ]:
api_key_index = 0
counter = 0

for player in list_ppuids_no_duplicates:
    counter += 1
    print('{} not in (list_master_player_ppuid or list_highDiamond_player_ppuid) = {}'.format(player, player not in (list_master_player_ppuid or list_highDiamond_player_ppuid)))
    if player not in (list_master_player_ppuid or list_highDiamond_player_ppuid):
        try:
            ppuid_info = get_name_and_tag_by_ppuid('americas', player, api_token_var_list[api_key_index])
            if ppuid_info == 400:
                print('wrong API key, cambiandola a otra ....')
                api_key_index +=1                
            elif ppuid_info == 429:
                api_key_index += 1
                print('Error en paso {}'.format(counter))
                print('Error 429, cambiando API Key a {}'.format(api_token_var_list[api_key_index]))
                if api_key_index > len(api_token_var_list)-1:
                    print('No me quedan más API Keys')
                    break
            elif match_info == 404:
                print('Error en paso {}'.format(counter))
                print('Error 404, no se encontro información relacionada al ppuid {}'.format(match))
            elif (df_master_leaderboard.iloc[:,0] == ppuid_info['gameName']).any() == True:
                print('Agregando {} a list_master_player_ppuid')
                list_master_player_ppuid.append(player)
            else:
                print('Agregando {} a list_highDiamond_player_ppuid')
                list_highDiamond_player_ppuid.append(player)
        except:
            print('algun error')

In [44]:
def writeData(filename, data):
    with open(filename, 'w') as file:
         file.write(format(json.dumps(data, ensure_ascii=False, indent=4)))

In [45]:
writeData('data/lor-player-data/master_player_list_ppuids.json', list_master_player_ppuid)
writeData('data/lor-player-data/high_diamond_player_ppuid_list.json', delete_duplicates_list(list_highDiamond_player_ppuid))

In [115]:
test = readData('data/lor-match-data/match-lists/match_list-20220211-233732.json')

In [117]:
len(delete_duplicates_list(test))

1598

In [46]:
master_player_list = readData('data/lor-player-data/master_player_list.json')

In [ ]:
master_player_list

In [48]:
new_d = []
for x in master_player_list:
    if x not in new_d:
        new_d.append(x)

In [50]:
len(new_d)

375

In [51]:
writeData('data/lor-player-data/master_player_list.json', new_d)

In [52]:
match1 = readData('data/lor-match-data/match-lists/match_list-20220211-233732.json')
match2 = readData('data/lor-match-data/match-lists/master-match_list-20220212-205559.json')

In [59]:
match1.index("755511c9-94d5-4349-8509-c1b0efe4bf82")

1063

In [69]:
len(delete_duplicates_list(match2))

6512

In [70]:
for x in match2:
    if x in match1[:1064]:
        match2.pop(match2.index(x))

In [72]:
writeData('data/lor-match-data/match-lists/master-match_list-20220212-232705.json', match2)

In [67]:
match1[:1064][1063]

'755511c9-94d5-4349-8509-c1b0efe4bf82'

In [57]:
match1[900]

'd9b210a3-d5ac-4fe9-80ed-1a8004a50ab4'

In [73]:
def test2():
    return 'a', 'b'

In [74]:
x = test2()

In [75]:
x[0]

'a'